In [ ]:

import pandas as pd

# 1. CSV 파일 불러오기
file_path = 'C:/Users/user/Desktop/병합 데이터/업종별_인천e음카드_데이터.csv'
df = pd.read_csv(file_path)

# 2. 날짜 컬럼 datetime으로 변환
df['일자'] = pd.to_datetime(df['일자'])

# 3. '월' 컬럼 만들기 (예: 2020-07)
df['일자'] = df['일자'].dt.to_period('M').astype(str)

# 4. 수치형 컬럼만 선택 (날짜, 지역명 제외)
numeric_cols = df.select_dtypes(include='number').columns.tolist()

# 5. 월 + 지역 기준으로 집계 (sum)
monthly_df = df.groupby(['일자', '지역'])[numeric_cols].sum().reset_index()
monthly_df = monthly_df.sort_values(by=['지역', '일자'])

# 6. 결과 확인
print(monthly_df.head())
monthly_df.to_csv('C:/Users/user/Desktop/병합 데이터/업종별_인천e음카드_데이터(월별)1.csv', index=False, encoding='utf-8-sig')

In [3]:
monthly_df = monthly_df.sort_values(by=['지역', '일자'])
monthly_df

,일자,지역,문화_취미_영화관,숙박,약국,연료,유통업영리_편의점,일반휴게음식,학원,기타,전체금액
0,2020-07,인천광역시 강화군,0.0,17919950.0,64674586.0,189703435.0,182899740.0,1.080620e+09,1.326047e+08,1.515162e+09,3.183584e+09
10,2020-08,인천광역시 강화군,0.0,21769840.0,57729479.0,177892028.0,185984670.0,1.081657e+09,1.192401e+08,1.496023e+09,3.140296e+09
20,2020-09,인천광역시 강화군,0.0,10618300.0,53659764.0,172369683.0,184410630.0,8.745608e+08,1.062951e+08,1.639904e+09,3.041818e+09
30,2020-10,인천광역시 강화군,0.0,13491690.0,54791504.0,193537522.0,198946707.0,1.148714e+09,1.420724e+08,1.679979e+09,3.431533e+09
40,2020-11,인천광역시 강화군,0.0,13625800.0,57317644.0,194349139.0,165580181.0,9.038431e+08,1.438935e+08,1.411647e+09,2.890256e+09
...,...,...,...,...,...,...,...,...,...,...,...
518,2024-10,인천광역시 중구,2166960.0,17673391.0,336149655.0,506199432.0,911472986.0,3.537901e+09,1.148462e+09,5.603171e+09,1.206320e+10
528,2024-11,인천광역시 중구,2071750.0,18000735.0,316041276.0,478201712.0,827862943.0,3.121756e+09,1.129418e+09,5.265322e+09,1.115867e+10
538,2024-12,인천광역시 중구,3547560.0,23206423.0,347182198.0,520460162.0,806038334.0,3.388644e+09,1.096078e+09,5.485112e+09,1.167027e+10
548,2025-01,인천광역시 중구,2153600.0,23418905.0,323104618.0,490758135.0,776510019.0,3.290804e+09,1.127757e+09,5.758079e+09,1.179259e+10


In [37]:
'''
import pandas as pd
import os
from functools import reduce

# 파일이 있는 폴더 경로
folder_path = 'C:/Users/user/Desktop/병합 데이터'
merge_keys = ['일자', '지역']

# 파일 이름과 인코딩 정보를 딕셔너리로 매핑
files_with_encoding = {
    "이용현황_결제건수_금액병합 수정.csv": "cp949",  
    "인천_행정구역별_월별_상권수 수정본.csv": "cp949",
    "인천e음카드_월별_캐시백비율.csv": "utf-8-sig",
    "인천광역시_행정동병_연령별_성별_인구수.csv": "cp949",
    "업종별_인천e음카드_데이터(월별).csv": "utf-8-sig"
}

dfs = []
for filename, encoding in files_with_encoding.items():
    path = os.path.join(folder_path, filename)
    
    # 파일 읽기
    df = pd.read_csv(path, encoding=encoding)

    
    # 날짜 컬럼 확인 및 변환
    if '일자' in df.columns:
        df['일자'] = pd.to_datetime(df['일자'])
    else:
        print(f"[경고] 파일 {filename}에 '일자' 컬럼 없음. 건너뜀.")
        continue

    # 지역 컬럼 확인
    if '지역' not in df.columns:
        print(f"[경고] 파일 {filename}에 '지역' 컬럼 없음. 건너뜀.")
        continue

    # 컬럼명 충돌 방지를 위해 prefix 붙이기
    prefix = os.path.splitext(filename)[0].replace(" ", "_")
    df = df.rename(columns={col: f"{prefix}_{col}" if col not in merge_keys else col for col in df.columns})
    
    dfs.append(df)

# 병합 (날짜 + 지역 기준)
merged_df = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='outer'), dfs)

# 정렬 및 저장
merged_df.sort_values(by=merge_keys, inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df = merged_df[merged_df['일자'] >= pd.Timestamp('2020-07-01')]
merged_df.to_csv('C:/Users/user/Desktop/병합 데이터/통합데이터(결측치처리X).csv', index=False)
merged_df

,일자,지역,이용현황_결제건수_금액병합_수정_충전건수,이용현황_결제건수_금액병합_수정_충전금액,이용현황_결제건수_금액병합_수정_결제건수,이용현황_결제건수_금액병합_수정_결제금액,인천_행정구역별_월별_상권수_수정본_상권수,인천e음카드_월별_캐시백비율_연매출 3억원 이하,인천e음카드_월별_캐시백비율_연매출 3억~30억원,인천e음카드_월별_캐시백비율_연매출 30억원 초과,...,인천광역시_행정동병_연령별_성별_인구수_남/총인구수,업종별_인천e음카드_데이터(월별)_문화_취미_영화관,업종별_인천e음카드_데이터(월별)_숙박,업종별_인천e음카드_데이터(월별)_약국,업종별_인천e음카드_데이터(월별)_연료,업종별_인천e음카드_데이터(월별)_유통업영리_편의점,업종별_인천e음카드_데이터(월별)_일반휴게음식,업종별_인천e음카드_데이터(월별)_학원,업종별_인천e음카드_데이터(월별)_기타,업종별_인천e음카드_데이터(월별)_전체금액
66,2020-07-01,인천광역시,894000.0,7.989800e+10,5032000.0,9.439900e+10,NaN,NaN,NaN,NaN,...,1475837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,2020-07-01,인천광역시 강화군,NaN,NaN,NaN,NaN,177.0,10%,10%,10%,...,34582,0.0,17919950.0,6.467459e+07,1.897034e+08,1.828997e+08,1.080620e+09,1.326047e+08,1.515162e+09,3.183584e+09
68,2020-07-01,인천광역시 계양구,55000.0,5.011000e+09,401000.0,7.340000e+09,159.0,10%,10%,10%,...,149306,6071400.0,38995050.0,7.341605e+08,9.837517e+08,1.183085e+09,6.286748e+09,1.421506e+09,1.023411e+10,2.088843e+10
69,2020-07-01,인천광역시 남동구,1000.0,8.500000e+07,52000.0,9.740000e+08,161.0,10%,10%,10%,...,263176,18542300.0,59330880.0,1.545309e+09,2.431556e+09,2.541397e+09,1.403066e+10,3.006268e+09,2.373890e+10,4.737196e+10
70,2020-07-01,인천광역시 동구,NaN,NaN,NaN,NaN,151.0,10%,10%,10%,...,31979,194900.0,2620000.0,1.749367e+08,2.811407e+08,2.302116e+08,9.700915e+08,2.274417e+08,2.063273e+09,3.949910e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,2025-02-01,인천광역시 부평구,189000.0,1.318900e+10,765000.0,1.445400e+10,95.0,10%,5%,0%,...,242667,749000.0,12752800.0,1.297974e+09,9.536321e+08,1.671684e+09,6.723649e+09,3.232600e+09,1.476464e+10,2.865768e+10
680,2025-02-01,인천광역시 서구,526000.0,3.967900e+10,2027000.0,4.288800e+10,111.0,10%,5%,0%,...,320820,4508750.0,12271804.0,1.343524e+09,1.697446e+09,2.355175e+09,9.754939e+09,7.371614e+09,1.900569e+10,4.154517e+10
681,2025-02-01,인천광역시 연수구,321000.0,2.574700e+10,1188000.0,2.800600e+10,168.0,10%,5%,0%,...,198313,1918590.0,11573398.0,7.677349e+08,1.458723e+09,1.303988e+09,5.806864e+09,5.393459e+09,1.251648e+10,2.726074e+10
682,2025-02-01,인천광역시 옹진군,NaN,NaN,NaN,NaN,103.0,10%,10%,0%,...,11237,0.0,1007500.0,3.407260e+06,5.350398e+07,4.766154e+07,1.004450e+08,2.746000e+06,1.376611e+08,3.464324e+08


In [5]:
import pandas as pd
df=pd.read_csv('C:/Users/user/Desktop/병합 데이터/인천e음_통합_월별 (2).csv',encoding='cp949')
df['월'] = pd.to_datetime(incheon_e['월'])
df['월'] = df['월'].dt.to_period('M').astype(str)

In [9]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

# ✅ LPF (Low-Pass Filter)
def apply_lpf(series, cutoff=0.05):
    b, a = butter(N=2, Wn=cutoff, btype='low')
    return filtfilt(b, a, series.fillna(method='ffill'))

# ✅ 지역별로 trend feature 생성
trend_features = []

for region, sub_df in df.groupby("지역"):
    temp = sub_df.copy()

    # --- (1) LPF 기반 추세
    temp["lpf_trend"] = apply_lpf(temp["전체금액"])

    # --- (2) 월별 추세 변화량
    temp["trend_diff"] = temp["lpf_trend"].diff()

    # --- (3) 최근 3개월간 기울기
    temp["slope_3"] = temp["lpf_trend"].rolling(3).apply(
        lambda x: np.polyfit(range(3), x, 1)[0] if not x.isnull().any() else np.nan,
        raw=False
    )

    # --- (4) 최근 6개월간 기울기
    temp["slope_6"] = temp["lpf_trend"].rolling(6).apply(
        lambda x: np.polyfit(range(6), x, 1)[0] if not x.isnull().any() else np.nan,
        raw=False
    )

    # --- (5) 실금액과 추세선 차이
    temp["trend_deviation"] = temp["전체금액"] - temp["lpf_trend"]

    trend_features.append(temp)

# ✅ 합치고 저장
df_trend = pd.concat(trend_features, ignore_index=True)
incheon_e["월"] = pd.to_datetime(incheon_e["월"])
df_trend["월"] = pd.to_datetime(df_trend["월"])

trend_cols = [
    "지역", "월", "lpf_trend", "trend_diff", "slope_3", "slope_6", "trend_deviation"
]
trend_only = df_trend[trend_cols]

# ✅ 병합 (left join)
incheon_e = pd.merge(incheon_e, trend_only, on=["지역", "월"], how="left")




C:\Users\user\AppData\Local\Temp\ipykernel_11428\2489473640.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return filtfilt(b, a, series.fillna(method='ffill'))
C:\Users\user\AppData\Local\Temp\ipykernel_11428\2489473640.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return filtfilt(b, a, series.fillna(method='ffill'))
C:\Users\user\AppData\Local\Temp\ipykernel_11428\2489473640.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return filtfilt(b, a, series.fillna(method='ffill'))
C:\Users\user\AppData\Local\Temp\ipykernel_11428\2489473640.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return filtfilt(b, a, series.fillna(m

In [96]:
# 날짜 오름차순 정렬 (같은 지역 내에서)
incheon_e['일자'] = pd.to_datetime(incheon_e['일자'])
incheon_e = incheon_e.sort_values(by=['지역', '일자'])

# 캐시백 변화 계산: diff()로 이전 달 대비 변화량 계산
incheon_e['캐시백_변화'] = incheon_e.groupby('지역')['인천e음카드_월별_캐시백비율_연매출 3억~30억원'].diff()

# 변화 방향을 나타내는 더미변수 생성
def get_change_direction(x):
    if pd.isna(x):
        return 0
    elif x > 0:
        return 1   # 상향
    elif x < 0:
        return -1  # 하향
    else:
        return 0   # 변화 없음

incheon_e['캐시백_상하향_더미'] = incheon_e['캐시백_변화'].apply(get_change_direction)

# 1. 기준이 되는 캐시백 비율 열 지정
cashback_col = '인천e음카드_월별_캐시백비율_연매출 3억~30억원'

# 2. 캐시백 변화 시점 추출
change_points = incheon_e[incheon_e['캐시백_상하향_더미'] != 0].copy()
change_points["캐시백_유지개월"] = 0  # 초기화

# 3. 각 변화 시점마다 이후 유지된 개월 수 계산
for idx, row in change_points.iterrows():
    지역 = row['지역']
    기준일자 = row['일자']
    기준비율 = row[cashback_col]

    # 해당 지역 데이터 중 기준일자 이후만 필터링
    after_df = incheon_e[(incheon_e['지역'] == 지역) & (incheon_e['일자'] > 기준일자)].copy()

    # 얼마나 유지됐는지 계산
    유지개월 = 0
    for _, r in after_df.iterrows():
        if r[cashback_col] == 기준비율:
            유지개월 += 1
        else:
            break

    # 결과 저장
    change_points.at[idx, "캐시백_유지개월"] = 유지개월

# 4. 결과를 원본 데이터프레임에 병합
incheon_e = pd.merge(
    incheon_e,
    change_points[["지역", "일자", "캐시백_유지개월"]],
    on=["지역", "일자"],
    how="left"
)


In [97]:
# 캐시백 이벤트가 발생한 행만 추출
event_df = incheon_e[incheon_e['캐시백_상하향_더미'] != 0].copy()

# 소비 변화율 계산 함수
def compute_cashback_effect(row):
    지역, 이벤트일자, 유지개월 = row['지역'], row['일자'], row['캐시백_유지개월']
    
    try:
        유지개월 = int(유지개월)
    except:
        return np.nan
    
    # 기준 월들: 이벤트 이전 n개월
    기준_월들 = pd.date_range(end=이벤트일자, periods=유지개월 + 1, freq='MS')[:-1]
    
    이전평균 = incheon_e.query("지역 == @지역 and 일자 in @기준_월들")['업종별_인천e음카드_데이터(월별)_전체금액'].mean()
    
    # 다음달
    다음달일자 = 이벤트일자 + pd.DateOffset(months=1)
    다음달금액 = incheon_e.query("지역 == @지역 and 일자 == @다음달일자")['업종별_인천e음카드_데이터(월별)_전체금액']
    
    if 다음달금액.empty or pd.isna(이전평균) or 이전평균 == 0:
        return np.nan
    
    return (다음달금액.values[0] - 이전평균) / 이전평균

# 계산 적용
event_df["캐시백_변동_소비변화율"] = event_df.apply(compute_cashback_effect, axis=1)

# 원본 df에 다시 병합 (중복 제거하고)
incheon_e = pd.merge(
    incheon_e,
    event_df[["지역", "일자", "캐시백_변동_소비변화율"]],
    on=["지역", "일자"],
    how="left"
)
incheon_e['일자'] = incheon_e['일자'].dt.to_period('M').astype(str)

In [101]:
incheon_e["일자"] = pd.to_datetime(incheon_e["일자"])
incheon_e["month"] = incheon_e["일자"].dt.month
incheon_e["month_sin"] = np.sin(2 * np.pi * incheon_e["month"] / 12)
incheon_e["month_cos"] = np.cos(2 * np.pi * incheon_e["month"] / 12)


In [110]:
# ✅ 1. 전월 대비 변화율 (%)
incheon_e["전월대비_변화율"] = incheon_e.groupby("지역")["업종별_인천e음카드_데이터(월별)_전체금액"].pct_change()

# ✅ 2. 3개월 이동평균과의 편차 (변동성)
incheon_e["금액_ma_3"] = incheon_e.groupby("지역")["업종별_인천e음카드_데이터(월별)_전체금액"].transform(lambda x: x.rolling(3, min_periods=1).mean())
incheon_e["금액_deviation_3"] = incheon_e["업종별_인천e음카드_데이터(월별)_전체금액"] - incheon_e["금액_ma_3"]

# ✅ 3. 연초/중반/연말 구분
incheon_e["month"] = incheon_e["일자"].dt.month
def classify_year_emt(month):
    if month <= 3:
        return "Y_E"  # 연초
    elif month <= 9:
        return "Y_M"  # 중반
    else:
        return "Y_T"  # 연말
incheon_e["연초말_EMT"] = incheon_e["month"].apply(classify_year_emt)

# 2. 지역 + 월별 평균 소비금액 계산
월별평균_df = (
    incheon_e
    .groupby(["지역", "month"])["업종별_인천e음카드_데이터(월별)_전체금액"]
    .mean()
    .reset_index()
    .rename(columns={"업종별_인천e음카드_데이터(월별)_전체금액": "월별평균"})
)

# 3. 병합해서 평년 대비 차이 계산
incheon_e = incheon_e.merge(월별평균_df, on=["지역", "month"], how="left")
incheon_e["평년_월별_차이"] = (
    incheon_e["업종별_인천e음카드_데이터(월별)_전체금액"] - incheon_e["월별평균"]
)

# 4. 필요하면 월별평균 컬럼 제거
incheon_e.drop(columns=["월별평균"], inplace=True)
incheon_e['일자'] = incheon_e['일자'].dt.to_period('M').astype(str)

In [112]:
incheon_e.to_csv('C:/Users/user/Desktop/병합 데이터/추세,계절성(월별).csv')